In [1]:
import os
import pandas as pd
import numpy as np

RES_DIR = "../res"

# Load experiment results

In [2]:
def load_data():
    df = pd.read_parquet(os.path.join(RES_DIR, "mnist_results.parquet"))
    npz_data = np.load(os.path.join(RES_DIR, "mnist_embeddings.npz"), allow_pickle=True)

    for key in ["distance", "original_image_pred_proba", "embedding_pred_proba", "modified_image_pred_proba", "modified_image"]:
        df[key] = list(npz_data[key])

    df.dropna(axis=1, inplace=True)
    df["input_name"] = df["input_name"].astype(str)
    return df

In [3]:
results = load_data()

In [4]:
results.head(10)

,dataset,delta_multiplier,patch_option,input_name,explored_patches,patch_attribution,number_explored_patches,save_each,algorithm,iteration,...,time,repetition,original_image_pred,embedding_pred,modified_image_pred,distance,original_image_pred_proba,embedding_pred_proba,modified_image_pred_proba,modified_image
0,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,0,...,1.579140,1,0,0,0,"[3.971209327417144e-22, -1.661561433792935e-21...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.74923, 0.23322332, 2.3773334, -3.7149956, ...","[17.74934, 0.23315895, 2.3776855, -3.7149475, ...","[0.99201345, 0.9918513, 0.9880024, 0.98821265,..."
1,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,10,...,1.421656,1,0,0,0,"[-4.70646143293721e-22, -2.0676721295379676e-2...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749186, 0.23323596, 2.3773115, -3.715068, ...","[17.748547, 0.2322985, 2.3775837, -3.7143686, ...","[0.9919369, 0.9923471, 0.98895395, 0.9880736, ..."
2,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,20,...,1.424537,1,0,0,0,"[-2.396311826487307e-21, 6.399177252262974e-21...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749182, 0.23321259, 2.3773406, -3.715109, ...","[17.750824, 0.23243535, 2.3765707, -3.7163098,...","[0.99226165, 0.99129015, 0.98828506, 0.9885561..."
3,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,30,...,1.424886,1,0,0,0,"[-7.599779965204836e-22, 1.2612495582200738e-2...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749176, 0.23321164, 2.3773093, -3.7151375,...","[17.751953, 0.23198068, 2.3749075, -3.7167404,...","[0.99099493, 0.9915606, 0.98898923, 0.9873359,..."
4,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,40,...,1.424292,1,0,0,0,"[8.090817080207663e-24, 3.475859864387167e-21,...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749147, 0.23322022, 2.377293, -3.7151904, ...","[17.755564, 0.23192537, 2.3736708, -3.7185116,...","[0.99141777, 0.99211276, 0.98964834, 0.9877171..."
5,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,50,...,1.407780,1,0,0,0,"[3.0408923416254107e-22, -1.3189565407973205e-...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.74916, 0.23321354, 2.3773668, -3.7151816, ...","[17.755424, 0.23248208, 2.3744967, -3.718568, ...","[0.9918133, 0.9918284, 0.9896072, 0.9880786, 0..."
6,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,60,...,1.411259,1,0,0,0,"[-1.8407725005499912e-22, 1.4558168356312028e-...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749123, 0.23325336, 2.377414, -3.7152104, ...","[17.75513, 0.23256147, 2.3745823, -3.7191308, ...","[0.99177015, 0.99264246, 0.99004066, 0.9881351..."
7,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,70,...,1.423276,1,0,0,0,"[1.817370456744504e-07, 2.693696097437075e-21,...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749113, 0.23322165, 2.377431, -3.7152588, ...","[17.755688, 0.23316944, 2.3756907, -3.7187102,...","[0.99258995, 0.9934547, 0.9910283, 0.9892013, ..."
8,mnist,1.0,all,img_8845,"[103, 52, 0, 90, 98, 180, 1, 160, 76, 175, 61,...","[0.04916961118578911, 0.025478344410657883, 0....",196,10,simec,80,...,1.433646,1,0,0,0,"[-1.2375636182662177e-21, -8.112502181555173e-...","[17.749237, 0.23321164, 2.3773289, -3.7149882,...","[17.749104,

In [5]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36360 entries, 0 to 36359
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dataset                    36360 non-null  object 
 1   delta_multiplier           36360 non-null  float64
 2   patch_option               36360 non-null  object 
 3   input_name                 36360 non-null  object 
 4   explored_patches           36360 non-null  object 
 5   patch_attribution          36360 non-null  object 
 6   number_explored_patches    36360 non-null  int64  
 7   save_each                  36360 non-null  int64  
 8   algorithm                  36360 non-null  object 
 9   iteration                  36360 non-null  int64  
 10  delta                      36360 non-null  float64
 11  time                       36360 non-null  float64
 12  repetition                 36360 non-null  int64  
 13  original_image_pred        36360 non-null  int

# Prediction probabilities

Is there a change in top prediction?

In [7]:
results["orig_vs_embed_diff"] = results["original_image_pred"] != results["embedding_pred"]
results["orig_vs_mod_diff"] = results["original_image_pred"] != results["modified_image_pred"]

# Group by the requested columns and aggregate the differences
grouped_res = results.groupby(
    ["dataset", "patch_option", "delta_multiplier", "algorithm"]
).agg(
    orig_vs_embed_diff_count=("orig_vs_embed_diff", "sum"),
    orig_vs_mod_diff_count=("orig_vs_mod_diff", "sum"),
    total=("original_image_pred", "count")
).reset_index()
grouped_res

,dataset,patch_option,delta_multiplier,algorithm,orig_vs_embed_diff_count,orig_vs_mod_diff_count,total
0,mnist,all,0.1,simec,0,0,3030
1,mnist,all,0.1,simexp,0,0,3030
2,mnist,all,1.0,simec,0,0,3030
3,mnist,all,1.0,simexp,0,0,3030
4,mnist,one,0.1,simec,0,0,3030
5,mnist,one,0.1,simexp,0,0,3030
6,mnist,one,1.0,simec,0,0,3030
7,mnist,one,1.0,simexp,0,0,3030
8,mnist,q2,0.1,simec,0,0,3030
9,mnist,q2,0.1,simexp,0,0,3030


Which are the images that change enought to cause a prediction shift? 

In [8]:
# Initialize a list to store the change data
change_records = []

# Define grouping columns
group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

# Group and compare
for group_keys, group in results.groupby(group_cols):
    group = group.sort_values("iteration")
    rows = list(group.itertuples(index=False))
    if not rows:
        continue

    prev_emb = rows[0].embedding_pred
    prev_mod = rows[0].modified_image_pred

    for row in rows[1:]:
        changed_emb = row.embedding_pred != prev_emb
        changed_mod = row.modified_image_pred != prev_mod

        if changed_emb or changed_mod:
            change_type = "Both" if changed_emb and changed_mod else ("Emb" if changed_emb else "Mod")

            change_records.append({
                "type": change_type,
                "dataset": row.dataset,
                "algorithm": row.algorithm,
                "delta_multiplier": row.delta_multiplier,
                "patch_option": row.patch_option,
                "input_name": row.input_name,
                "repetition": row.repetition,
                "iteration": row.iteration,
                "original_pred": row.original_image_pred,
                "embedding_pred": row.embedding_pred,
                "modified_pred": row.modified_image_pred
            })

        prev_emb = row.embedding_pred
        prev_mod = row.modified_image_pred

# Convert to DataFrame
df_pred_changes = pd.DataFrame(change_records)

# Show result
df_pred_changes

""


At which iteration does the change occur?

In [ ]:
# Expanded summary statistics for iteration changes
summary_stats = (
    df_pred_changes
    .groupby(["type", "algorithm", "patch_option", "delta_multiplier"])["iteration"]
    .agg(
        count="count",
        mean="mean",
        std="std",
        min="min",
        max="max",
        median="median",
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75)
    )
    .reset_index()
    .sort_values(["patch_option", "algorithm", "delta_multiplier"])
)

# Display the summary
summary_stats

,type,algorithm,patch_option,delta_multiplier,count,mean,std,min,max,median,q25,q75
2,Emb,simexp,all,0.1,17,315.294118,327.626267,10,810,130.0,20.0,650.0
6,Mod,simexp,all,0.1,11,410.000000,405.462699,30,970,100.0,65.0,745.0
3,Emb,simexp,all,1.0,18,392.222222,288.496214,90,950,275.0,185.0,530.0
1,Emb,simec,q2,1.0,9,341.111111,79.599693,280,540,320.0,300.0,340.0
4,Emb,simexp,q2,0.1,9,343.333333,200.686322,100,580,370.0,100.0,530.0
0,Both,simexp,q2,1.0,1,70.000000,NaN,70,70,70.0,70.0,70.0
5,Emb,simexp,q2,1.0,33,406.969697,255.128458,10,750,480.0,180.0,620.0
7,Mod,simexp,q2,1.0,6,106.666667,114.833212,40,340,65.0,60.0,70.0


Which are the images where the top class has an increasing probability (and viceversa)?

In [9]:
records = []  # List to collect results for DataFrame

group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

for group_keys, group in results.groupby(group_cols):
    group = group.sort_values("iteration")
    rows = list(group.itertuples(index=False))
    if not rows:
        continue

    # Initialize previous preds
    original_pred = rows[0].original_image_pred
    prev_proba_original_pred = rows[0].embedding_pred_proba[original_pred]
    prev_change_type = "="

    # Iterate from second row onward
    for row in rows[1:]:
        curr_proba = row.embedding_pred_proba[original_pred]
        if curr_proba < prev_proba_original_pred:
            change_type = "-"
        elif curr_proba > prev_proba_original_pred:
            change_type = "+"
        else:
            change_type = "="

        if change_type != prev_change_type:
            diff = curr_proba - prev_proba_original_pred
            records.append({
                "type": change_type,
                "dataset": row.dataset,
                "algorithm": row.algorithm,
                "delta_multiplier": row.delta_multiplier,
                "patch_option": row.patch_option,
                "input_name": row.input_name,
                "repetition": row.repetition,
                "iteration": row.iteration,
                "diff": diff,
            })

            prev_change_type = change_type

        # Update trackers
        prev_proba_original_pred = curr_proba

# Create DataFrame from collected records
df_changes = pd.DataFrame.from_records(records)
df_changes

,type,dataset,algorithm,delta_multiplier,patch_option,input_name,repetition,iteration,diff
0,+,mnist,simec,0.1,all,img_1261,1,20,0.000004
1,-,mnist,simec,0.1,all,img_1261,1,50,-0.000004
2,=,mnist,simec,0.1,all,img_1261,1,70,0.000000
3,+,mnist,simec,0.1,all,img_1261,1,80,0.000004
4,-,mnist,simec,0.1,all,img_1261,1,90,-0.000004
...,...,...,...,...,...,...,...,...,...
16428,+,mnist,simexp,1.0,q2,img_9718,3,920,0.000623
16429,-,mnist,simexp,1.0,q2,img_9718,3,930,-0.005704
16430,+,mnist,simexp,1.0,q2,img_9718,3,940,0.006650
16431,-,mnist,simexp,1.0,q2,img_9718,3,960,-0.000756


In [10]:
group_cols = ["patch_option", "delta_multiplier", "algorithm", "dataset", "input_name"]
# --- Determine majority change per group ---
majority_summary = (
    df_changes
    .groupby(group_cols + ["type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Determine majority type
majority_summary["majority_type"] = majority_summary[["+", "-", "="]].idxmax(axis=1)

# Optional: Rename columns for clarity
majority_summary = majority_summary.rename(columns={"+": "count_+", "-": "count_-", "=": "count_="})

# Show the result
majority_summary

type,patch_option,delta_multiplier,algorithm,dataset,input_name,count_+,count_-,count_=,majority_type
0,all,0.1,simec,mnist,img_1261,84,74,36,+
1,all,0.1,simec,mnist,img_1367,84,82,33,+
2,all,0.1,simec,mnist,img_3737,67,47,24,+
3,all,0.1,simec,mnist,img_4341,56,70,19,-
4,all,0.1,simec,mnist,img_6943,87,81,31,+
...,...,...,...,...,...,...,...,...,...
115,q2,1.0,simexp,mnist,img_6943,60,60,0,+
116,q2,1.0,simexp,mnist,img_7373,62,60,0,+
117,q2,1.0,simexp,mnist,img_8599,61,63,0,-
118,q2,1.0,simexp,mnist,img_8987,58,60,0,-


In [11]:
records = []  # List to collect results for DataFrame

group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

for group_keys, group in results.groupby(group_cols):
    group = group.sort_values("iteration")
    rows = list(group.itertuples(index=False))
    if not rows:
        continue

    # Initialize previous preds
    original_pred = rows[0].original_image_pred
    prev_proba_original_pred = rows[0].embedding_pred_proba[original_pred]
    prev_change_type = "="

    # Iterate from second row onward
    for row in rows[1:]:
        curr_proba = row.embedding_pred_proba[original_pred]
        if curr_proba < prev_proba_original_pred:
            change_type = "-"
        elif curr_proba > prev_proba_original_pred:
            change_type = "+"
        else:
            change_type = "="

        diff = curr_proba - prev_proba_original_pred
        records.append({
            "type": change_type,
            "dataset": row.dataset,
            "algorithm": row.algorithm,
            "delta_multiplier": row.delta_multiplier,
            "patch_option": row.patch_option,
            "input_name": row.input_name,
            "repetition": row.repetition,
            "iteration": row.iteration,
            "diff": diff,
        })

        prev_change_type = change_type

        # Update trackers
        prev_proba_original_pred = curr_proba

# Create DataFrame from collected records
df_changes = pd.DataFrame.from_records(records)
# Define grouping columns (experiment-level identifiers)
group_cols = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name", "repetition"]

# Assume each experiment has 100 iterations saved (one every 10 iterations for 1000 total iterations)
total_iterations = 101

# Group df_changes by the experiment identifiers plus the change type and count the number of unique iterations
change_summary = (
    df_changes.groupby(group_cols + ["type"])["iteration"]
    .nunique()  # count unique iterations where a change was recorded
    .reset_index(name="change_count")
)

# Pivot the table so that each type ("+", "-", "=") becomes its own column
pivot_summary = change_summary.pivot_table(
    index=group_cols,
    columns="type",
    values="change_count",
    fill_value=0
).reset_index()

# Rename the columns for clarity if present
if "+" in pivot_summary.columns:
    pivot_summary = pivot_summary.rename(columns={"+": "increase_count"})
if "-" in pivot_summary.columns:
    pivot_summary = pivot_summary.rename(columns={"-": "decrease_count"})
if "=" in pivot_summary.columns:
    pivot_summary = pivot_summary.rename(columns={"=": "equal_count"})

# In case a type is missing, add it with 0
for col in ["increase_count", "decrease_count", "equal_count"]:
    if col not in pivot_summary.columns:
        pivot_summary[col] = 0

# Calculate percentage for each change type per experiment
pivot_summary["pct_increase"] = (pivot_summary["increase_count"] / total_iterations) * 100
pivot_summary["pct_decrease"] = (pivot_summary["decrease_count"] / total_iterations) * 100
pivot_summary["pct_equal"] = (pivot_summary["equal_count"] / total_iterations) * 100

# Display the final summary DataFrame
pivot_summary

type,dataset,patch_option,delta_multiplier,algorithm,input_name,repetition,increase_count,decrease_count,equal_count,pct_increase,pct_decrease,pct_equal
0,mnist,all,0.1,simec,img_1261,1,42.0,46.0,12.0,41.584158,45.544554,11.881188
1,mnist,all,0.1,simec,img_1261,2,53.0,30.0,17.0,52.475248,29.702970,16.831683
2,mnist,all,0.1,simec,img_1261,3,44.0,40.0,16.0,43.564356,39.603960,15.841584
3,mnist,all,0.1,simec,img_1367,1,38.0,53.0,9.0,37.623762,52.475248,8.910891
4,mnist,all,0.1,simec,img_1367,2,44.0,41.0,15.0,43.564356,40.594059,14.851485
...,...,...,...,...,...,...,...,...,...,...,...,...
355,mnist,q2,1.0,simexp,img_8987,2,30.0,70.0,0.0,29.702970,69.306931,0.000000
356,mnist,q2,1.0,simexp,img_8987,3,47.0,53.0,0.0,46.534653,52.475248,0.000000
357,mnist,q2,1.0,simexp,img_9718,1,7.0,93.0,0.0,6.930693,92.079208,0.000000
358,mnist,q2,1.0,simexp,img_9718,2,15.0,85.0,0.0,14.851485,84.158416,0.000000


In [57]:
pivot_summary[pivot_summary["input_name"] == "img_1248"]

type,dataset,patch_option,delta_multiplier,algorithm,input_name,repetition,increase_count,decrease_count,equal_count,pct_increase,pct_decrease,pct_equal
0,cifar,all,0.1,simec,img_1248,1,64.0,35.0,1.0,63.366337,34.653465,0.990099
1,cifar,all,0.1,simec,img_1248,2,55.0,45.0,0.0,54.455446,44.554455,0.000000
2,cifar,all,0.1,simec,img_1248,3,64.0,36.0,0.0,63.366337,35.643564,0.000000
60,cifar,all,0.1,simexp,img_1248,1,0.0,100.0,0.0,0.000000,99.009901,0.000000
61,cifar,all,0.1,simexp,img_1248,2,5.0,95.0,0.0,4.950495,94.059406,0.000000
62,cifar,all,0.1,simexp,img_1248,3,1.0,99.0,0.0,0.990099,98.019802,0.000000
120,cifar,all,1.0,simec,img_1248,1,40.0,60.0,0.0,39.603960,59.405941,0.000000
121,cifar,all,1.0,simec,img_1248,2,41.0,59.0,0.0,40.594059,58.415842,0.000000
122,cifar,all,1.0,simec,img_1248,3,39.0,60.0,1.0,38.613861,59.405941,0.990099
180,cifar,all,1.0,simexp,img_1248,1,16.0,84.0,0.0,15.841584,83.168317,0.000000


In [13]:
high_level_summary = (
    pivot_summary
    .groupby(["algorithm", "patch_option", "delta_multiplier"])
    .agg(
        mean_pct_increase=("pct_increase", "mean"),
        mean_pct_decrease=("pct_decrease", "mean"),
        mean_pct_equal=("pct_equal", "mean"),
        std_pct_increase=("pct_increase", "std"),
        std_pct_decrease=("pct_decrease", "std"),
        std_pct_equal=("pct_equal", "std"),
        num_experiments=("pct_equal", "count")  # or any column, since they're all same length
    )
    .reset_index()
)

# Show the summarized result
high_level_summary

,algorithm,patch_option,delta_multiplier,mean_pct_increase,mean_pct_decrease,mean_pct_equal,std_pct_increase,std_pct_decrease,std_pct_equal,num_experiments
0,simec,all,0.1,46.138614,42.838284,10.033003,12.882050,13.410038,3.387491,30
1,simec,all,1.0,48.613861,48.910891,1.485149,20.793461,20.328607,1.532752,30
2,simec,one,0.1,39.669967,36.963696,22.376238,5.359879,4.430401,6.150827,30
3,simec,one,1.0,41.188119,38.415842,19.405941,10.386199,7.420164,5.562085,30
4,simec,q2,0.1,54.554455,33.597360,10.858086,14.633313,13.268125,5.375938,30
5,simec,q2,1.0,50.792079,45.478548,2.739274,16.873104,16.430117,2.406816,30
6,simexp,all,0.1,51.980198,46.996700,0.033003,27.535275,27.556117,0.180767,30
7,simexp,all,1.0,38.481848,60.528053,0.000000,16.095048,16.095048,0.000000,30
8,simexp,one,0.1,59.207921,39.174917,0.627063,32.844457,32.830337,0.989530,30
9,simexp,one,1.0,58.481848,40.495050,0.033003,30.490231,30.452976,0.180767,30


In [39]:
majority_summary[majority_summary["input_name"] == "img_1248"]

type,patch_option,delta_multiplier,algorithm,dataset,input_name,count_+,count_-,count_=,majority_type
0,all,0.1,simec,cifar,img_1248,75,74,1,+
20,all,0.1,simexp,cifar,img_1248,4,6,0,-
40,all,1.0,simec,cifar,img_1248,69,70,1,-
60,all,1.0,simexp,cifar,img_1248,40,42,0,-
80,one,0.1,simec,cifar,img_1248,80,79,60,+
100,one,0.1,simexp,cifar,img_1248,4,6,0,-
121,one,1.0,simec,cifar,img_1248,80,79,46,+
141,one,1.0,simexp,cifar,img_1248,71,73,1,-
160,q2,0.1,simec,cifar,img_1248,81,77,4,+
180,q2,0.1,simexp,cifar,img_1248,0,3,0,-


In [14]:
# Define the group columns to flatten over iterations and repetitions
group_cols_flat = ["dataset", "patch_option", "delta_multiplier", "algorithm", "input_name"]

# Count occurrences of each change type
type_counts = (
    df_changes
    .groupby(group_cols_flat + ["type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "- count", "+": "+ count", "=": "= count"})
)

# Compute mean diff for each change type
type_means = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .mean()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "mean -", "+": "mean +", "=": "mean ="})
)

# Compute sumn diff for each change type
type_sums = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .sum()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "sum -", "+": "sum +", "=": "sum ="})
)

# Merge counts and means
summary_df = pd.merge(type_counts, type_means, on=group_cols_flat)

# Merge the result with sums
summary_df = pd.merge(summary_df, type_sums, on=group_cols_flat)
pd.set_option('display.max_rows', 500)
# Display the final summary
summary_df

type,dataset,patch_option,delta_multiplier,algorithm,input_name,+ count,- count,= count,mean +,mean -,mean =,sum +,sum -,sum =
0,mnist,all,0.1,simec,img_1261,139,116,45,5.118281e-06,-4.455961e-06,0.0,0.000711,-0.000517,0.0
1,mnist,all,0.1,simec,img_1367,127,137,36,6.593119e-06,-5.763813e-06,0.0,0.000837,-0.000790,0.0
2,mnist,all,0.1,simec,img_3737,217,56,27,6.249423e-06,-3.082412e-06,0.0,0.001356,-0.000173,0.0
3,mnist,all,0.1,simec,img_4341,80,199,21,3.838539e-06,-5.928116e-06,0.0,0.000307,-0.001180,0.0
4,mnist,all,0.1,simec,img_6943,135,128,37,4.803693e-06,-5.677342e-06,0.0,0.000648,-0.000727,0.0
5,mnist,all,0.1,simec,img_7373,158,112,30,3.790554e-06,-3.167561e-06,0.0,0.000599,-0.000355,0.0
6,mnist,all,0.1,simec,img_7819,174,91,35,4.883470e-06,-3.667978e-06,0.0,0.000850,-0.000334,0.0
7,mnist,all,0.1,simec,img_8257,133,142,25,3.757333e-06,-3.989314e-06,0.0,0.000500,-0.000566,0.0
8,mnist,all,0.1,simec,img_8434,143,142,15,6.875792e-06,-6.266043e-06,0.0,0.000983,-0.000890,0.0
9,mnist,all,0.1,simec,img_8784,92,175,33,4.001286e-06,-5.525862e-06,0.0,0.000368,-0.000967,0.0


In [15]:
# Define the group columns to flatten over iterations and repetitions
group_cols_flat = ["patch_option", "delta_multiplier", "algorithm"]

# Count occurrences of each change type
type_counts = (
    df_changes
    .groupby(group_cols_flat + ["type"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "- count", "+": "+ count", "=": "= count"})
)

# Compute mean diff for each change type
type_means = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .mean()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "mean -", "+": "mean +", "=": "mean ="})
)

# Compute sumn diff for each change type
type_sums = (
    df_changes
    .groupby(group_cols_flat + ["type"])["diff"]
    .sum()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"-": "sum -", "+": "sum +", "=": "sum ="})
)

# Merge counts and means
summary_df = pd.merge(type_counts, type_means, on=group_cols_flat)

# Merge the result with sums
summary_df = pd.merge(summary_df, type_sums, on=group_cols_flat)

# Display the final summary
summary_df

summary_df

type,patch_option,delta_multiplier,algorithm,+ count,- count,= count,mean +,mean -,mean =,sum +,sum -,sum =
0,all,0.1,simec,1398,1298,304,0.000005,-0.000005,0.0,0.007160,-0.006497,0.0
1,all,0.1,simexp,1575,1424,1,0.000813,-0.000841,0.0,1.281199,-1.197781,0.0
2,all,1.0,simec,1473,1482,45,0.000046,-0.000038,0.0,0.067217,-0.056705,0.0
3,all,1.0,simexp,1166,1834,0,0.006425,-0.008584,0.0,7.491294,-15.742599,0.0
4,one,0.1,simec,1202,1120,678,0.000002,-0.000002,0.0,0.002488,-0.001976,0.0
5,one,0.1,simexp,1794,1187,19,0.000068,-0.000060,0.0,0.122057,-0.071803,0.0
6,one,1.0,simec,1248,1164,588,0.000004,-0.000002,0.0,0.005452,-0.002586,0.0
7,one,1.0,simexp,1772,1227,1,0.000669,-0.000638,0.0,1.185257,-0.782666,0.0
8,q2,0.1,simec,1653,1018,329,0.000004,-0.000003,0.0,0.007163,-0.003477,0.0
9,q2,0.1,simexp,1459,1541,0,0.000604,-0.000625,0.0,0.881777,-0.963484,0.0
